In this notebook we first extracted table of contents. Then by using table of contents as guides, we separeted the whole reports into artitcles. Since we are only interested in Recommendation sections, Recommendation reports are selected. There are many customized search for different years of reports. The final output of this notebook is a table with all the recommendations by years. Rare special characters are removed. Spells are corrected.

In [0]:
%run ./utilities/Install_Spacy

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
%run ./utilities/Data_Clean_Functions

Out[4]: True

In [0]:
import numpy as np
import pandas as pd
import re

In [0]:
# read CMV reports file 
# "CMV_Reports_Text.csv" file has PDF scaned CMV reports data
# "CMV_Report_Dates.csv" file has two columns. One column is Year, another column is the first page number of its corresponding year of report. It is used for labeling years to their corresponding articles.
reports = pd.read_csv('/dbfs/FileStore/CMV_Reports_Text.csv')
report_dates = pd.read_csv('/dbfs/FileStore/CMV_Report_Dates.csv')

In [0]:
report_dates.head()

,PageNumber,Year
0,1,1996
1,59,1997
2,82,1998
3,120,1999
4,148,2000


In [0]:
# create the pageNumber and Year for the last page of the report
lastrow = pd.DataFrame({'PageNumber': reports.sort_values('PageNumber').iloc[-1]['PageNumber'], 
                                                      'Year': report_dates.iloc[-1]['Year']}, index=[0])

In [0]:
report_dates = pd.concat([report_dates, lastrow]).reset_index(drop=True)

In [0]:
report_dates.tail()

,PageNumber,Year
24,842,2016
25,885,2017
26,926,2018
27,973,2020
28,1016,2020


In [0]:
# this function created PageNumber and Years for pages in between pages in report_dates table
def create_rows_between(report_dates):
    report_between = pd.DataFrame(columns = ['PageNumber', 'Year'])
    years  = []
    pagenums = []
    for i in range(1, len(report_dates)):
        between_n = report_dates['PageNumber'][i] - report_dates['PageNumber'][i-1] -1
        years.extend(np.repeat([report_dates['Year'][i-1]], between_n ))
        
        prev = report_dates['PageNumber'][i-1]
        for k in range(between_n):
            curr = prev + 1
            pagenums.append(curr)
            prev = curr
            
    report_between['Year'] = years 
    report_between['PageNumber'] = pagenums 
    
    return report_between
report_between = create_rows_between(report_dates)

In [0]:
report_between.head()

,PageNumber,Year
0,2,1996
1,3,1996
2,4,1996
3,5,1996
4,6,1996


In [0]:
report_between.tail()

,PageNumber,Year
982,1011,2020
983,1012,2020
984,1013,2020
985,1014,2020
986,1015,2020


In [0]:
# get all the pages and their year labels
report_dates = pd.concat([report_dates, report_between])
# merge report text with dates
reports = reports.merge(report_dates, how = 'outer').reset_index(drop=True)

In [0]:
reports.head()

,PageNumber,ReportText,IsTableOfContents,IsCoverPage,DocumentBegin,IsAttatchment,Year
0,1,Ne eae |\nTHE SECOND ANNUAL REPORT\n/ OF THE\n...,0,1,1,0,1996
1,2,is DEPARTMENT OF VETERANS AFFAIRS\ny “RY % a :...,1,0,0,0,1996
2,3,"i a\nStatement of the Chairman, Advisory Commi...",1,0,0,0,1996
3,4,A great many people throughout the Congress of...,1,0,0,0,1996
4,5,"TABLE OF CONTENTS\nStatement of the Chairman, ...",0,0,0,0,1996


In [0]:
# clean up text
reports['ReportText'] = [remove_extra_space(
    remove_rare_special_character(
        fix_typo(
            remove_lo(
                remove_vertical_line(
                    fix_roma_numbers(
                        remove_extra_dot(
                            remove_hyphen(
                                remove_colon(
                                    remove_quotes(
                                        replace_quotes(
                                            remove_bracket(
                                                normalize_week(
                                                    normalize_month(
                                                        normalize_year(
                                                            i.lower()))))))))))))))).strip() for i in reports['ReportText']]

In [0]:
# get the index of table of contents rows
TOC_idx = [i for i, s in enumerate(reports['ReportText']) if re.findall("TABLE OF CONTENTs?( ?)+(page)?\n".lower(), s ) ]

In [0]:
# there should be 23 "table of contents"
reports_TOC = reports.iloc[TOC_idx]
print(len(reports_TOC))

21


In [0]:
# adding table of content page for 2016
reports_TOC = pd.concat([reports_TOC, reports[reports['PageNumber'] == 844]]).reset_index(drop=True)
# now we are only missing 2010 and 2019 which is true in the original doc

In [0]:
# add table of content manually for 2001
reports_TOC = pd.concat([reports_TOC, pd.DataFrame({'PageNumber':190,
'ReportText':'table of contents\nintroduction\ngeneral overview\nReport on the wilmington, de, meeting\ngeneral\nveterans entrepreneurship\nVeterans Entrepreneurship and Small Business Development\nWorkforce Diversity Plan\nGeneral\nSecretary Principis Comments\nVeterans Benefit Clearinghouse VBC \nVocational Rehabilitation & Employment Service\nServing Native American Veterans\nHepatitis C Treatment\nCenter for Minority Veterans\nCommittee Recommendations\nAppendix A\nAppendix B\nAdvisory Committee Agenda Continued\nAppendix C\n'.lower(),
'IsTableOfContents': 1,
'IsCoverPage': 0,
'DocumentBegin': 1,
'IsAttatchment': 0,
'Year': 2001}, index = [0])]).reset_index(drop=True)

In [0]:
len(reports_TOC)

Out[92]: 23

In [0]:
# reports' table of content
reports_TOC.head()

,PageNumber,ReportText,IsTableOfContents,IsCoverPage,DocumentBegin,IsAttatchment,Year,IsTableOfContents1
0,5,"table of contents\nstatement of the chairman, ...",0,0,0,0,1996,1
1,60,table of contents\npage\nstatement of the chai...,1,0,0,0,1997,1
2,83,table of contents\npage\n\nchairmans transmitt...,1,0,0,0,1998,1
3,121,table of contents\npage\nfifth annual report 1...,1,0,0,0,1999,1
4,152,table of contents\n1 letter of transmittal\n2 ...,1,0,0,0,2000,1


In [0]:
reports_TOC['IsTableOfContents1'] = 1
reports = reports.merge(reports_TOC[['PageNumber', 'IsTableOfContents1']], how = 'left').reset_index(drop=True)
import math
reports['IsTableOfContents1'] = [0 if math.isnan(i) else int(i) for i in reports['IsTableOfContents1']]

In [0]:
reports.head()

,PageNumber,ReportText,IsTableOfContents,IsCoverPage,DocumentBegin,IsAttatchment,Year,IsTableOfContents1
0,1,ne eae \nthe second annual report\n of the\nad...,0,1,1,0,1996,0
1,2,is department of veterans affairs\ny ry a cent...,1,0,0,0,1996,0
2,3,"i a\nstatement of the chairman, advisory commi...",1,0,0,0,1996,0
3,4,a great many people throughout the congress of...,1,0,0,0,1996,0
4,5,"table of contents\nstatement of the chairman, ...",0,0,0,0,1996,1


In [0]:
# separate reports into articles based on table of contents
articleDF = pd.DataFrame(columns = ['Title', 'ArticleText', 'PageNumber'])
for k in range(len(reports_TOC)):
    articleDF = pd.concat([articleDF, separate_text_into_articles(reports_TOC, k)])

In [0]:
articleDF.head()

,Title,ArticleText,PageNumber
0,Begin of the report,ne eae \nthe second annual report\n of the\nad...,"[1, 2, 3]"
1,"statement of the chairman, advisory committee ...","\npursuant to section 510, public law 103 446,...","[3, 4, 5, 6]"
2,executive summary,\nthe second annual report of the advisory com...,"[6, 7, 8]"
3,introduction,"\nthe committee conducted its last session, mo...",[8]
4,general,\nthe subcommittees based their pian on three ...,"[8, 9]"


In [0]:
# separate articles even more by "/n/n" which can be seen as the separation symbol for paragraphs
# first set unique id for each article
# regeneate index for articles
articleDF = articleDF.reset_index(drop = True)
# put index to column
articleDF = articleDF.reset_index(drop = False)
articleDF = articleDF.rename(columns = {'index': 'ArticleIndex'})

In [0]:
# add year column
articleDF['Year'] = [reports[reports['PageNumber'] == i[-1]]['Year'].tolist()[0] for i in articleDF['PageNumber']]

In [0]:
# select recommendations
recommd = articleDF[['recommendation' in i for i in articleDF['Title']]]

In [0]:
# check if I get all the recommdations from all years
recommd.Year.unique()
recommd = recommd.reset_index(drop = True)

Out[103]: array([1996, 1997, 1998, 1999, 2000, 2002, 2004, 2005, 2006, 2007, 2008,
       2009, 2011, 2012, 2013, 2014, 2015, 2017, 2018, 2020, 2016, 2001])

In [0]:
recommd.head()

,ArticleIndex,Title,ArticleText,PageNumber,Year
0,8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
1,23,recommendations,the full committee recommends that the secreta...,"[67, 68, 69, 70, 71, 72, 73]",1997
2,37,statement of recommendations,\n1. the full committee recommends the secreta...,"[94, 95]",1998
3,39,recommendations,\n1. initiate a review of the funding and oper...,"[98, 99, 100]",1998
4,43,vas response to advisory committee recommendat...,\n1. initiate a review of the funding and oper...,"[113, 114, 115, 116, 117, 118, 119]",1998


In [0]:
# missing 2011, 2012, 2003
# there is no recommendation reports in 2003. But there are Issues mentioned there
# fixed 2011 in the raw csv file. table of content part I was scaned as part | ,and in the article it is scaned as l
# fixed 2012. It is similar case with 2011.
# since 2008, the format of recommendation change from "recommendations" only to "recommendations and va responses" to "recommendations, rationales and VA Responses"

In [0]:
# need to fix recommendations that are not captured correctly.
recommd = recommd[[('rationale' not in str(recommd['Title'][i])) or (recommd['Year'][i] in (2009, 2011, 2012)) for i in range(len(recommd)) ]]

In [0]:
# fix 2013
rec_strt = re.search('acmv year recommendations, rationales, and va responses', recommd[recommd['Year'] == 2013]['ArticleText'].tolist()[0]).span()[0]-13
recommd.loc[recommd['Year'] == 2013, 'ArticleText'] = recommd[recommd['Year'] == 2013]['ArticleText'].tolist()[0][:rec_strt]

/databricks/python/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [0]:
# fix 2008
# index for recommendation
rec_idx = []
prev = 0
for i in re.finditer('recommendation \d+', recommd[recommd['Year'] == 2008]['ArticleText'].tolist()[0]):
    curr = int(re.findall('\d+', i.group(0))[0])
    if curr>prev:
        rec_idx.append(i.span()[0])
        prev = curr
# ignore the match that has occured before

In [0]:
len(rec_idx)

Out[112]: 22

In [0]:
# index for the start of other content other than recommendations
va_idx = []
prev = 0
for i in re.finditer('va response', recommd[recommd['Year'] == 2008]['ArticleText'].tolist()[0]):
    va_idx.append(i.span()[0])
   

In [0]:
len(va_idx)

Out[113]: 22

In [0]:
rec_str = []
recommd2008 = recommd[recommd['Year'] == 2008]['ArticleText'].tolist()[0]
for i in range(len(rec_idx)):
    rec_str.append(recommd2008[rec_idx[i]:va_idx[i]])

In [0]:
recommd.loc[recommd['Year'] == 2008, 'ArticleText'] = ''.join(rec_str)

/databricks/python/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [0]:
# fix 2009
# index for recommendation
rec_idx = []
prev = 0
for i in re.finditer(r'\n\d+\. ', recommd[recommd['Year'] == 2009]['ArticleText'].tolist()[0]):
    curr = int(re.findall(r'\d+', i.group(0))[0])
    if curr>prev:
        rec_idx.append(i.span()[0])
        prev = curr
# ignore the match that has occured before

# index for the start of other content other than recommendations
va_idx = []
prev = 0
for i in re.finditer(r'\nrationale', recommd[recommd['Year'] == 2009]['ArticleText'].tolist()[0]):
    va_idx.append(i.span()[0])

In [0]:
len(rec_idx)

Out[117]: 14

In [0]:
len(va_idx)

Out[118]: 14

In [0]:
rec_str = []
recommd2009 = recommd[recommd['Year'] == 2009]['ArticleText'].tolist()[0]
for i in range(len(rec_idx)):
    rec_str.append(recommd2009[rec_idx[i]:va_idx[i]])

In [0]:
recommd.loc[recommd['Year'] == 2009, 'ArticleText'] = ''.join(rec_str)

In [0]:
# fix 2011
# index for recommendation
rec_idx = []
prev = 0
for i in re.finditer(r'\n\d+\. ', recommd[recommd['Year'] == 2011]['ArticleText'].tolist()[0]):
    curr = int(re.findall(r'\d+', i.group(0))[0])
    if curr>prev:
        rec_idx.append(i.span()[0])
        prev = curr
# ignore the match that has occured before

# index for the start of other content other than recommendations
va_idx = []
prev = 0
for i in re.finditer(r'\nrationale', recommd[recommd['Year'] == 2011]['ArticleText'].tolist()[0]):
    va_idx.append(i.span()[0])

In [0]:
print(len(rec_idx))
print(len(va_idx))

13
13


In [0]:
rec_str = []
recommd2011 = recommd[recommd['Year'] == 2011]['ArticleText'].tolist()[0]
for i in range(len(rec_idx)):
    rec_str.append(recommd2011[rec_idx[i]:va_idx[i]])
    
recommd.loc[recommd['Year'] == 2011, 'ArticleText'] = ''.join(rec_str)

In [0]:
# fix 2012
# index for recommendation
rec_idx = []
prev = 0
for i in re.finditer(r'recommendation \d+ ', recommd[recommd['Year'] == 2012]['ArticleText'].tolist()[0]):
    curr = int(re.findall(r'\d+', i.group(0))[0])
    if curr>prev:
        rec_idx.append(i.span()[0])
        prev = curr
# ignore the match that has occured before

# index for the start of other content other than recommendations
va_idx = []
prev = 0
for i in re.finditer(r'\nrationale \d+ ', recommd[recommd['Year'] == 2012]['ArticleText'].tolist()[0]):
    va_idx.append(i.span()[0])

In [0]:
print(len(rec_idx))
print(len(va_idx))

11
11


In [0]:
rec_str = []
recommd2012 = recommd[recommd['Year'] == 2012]['ArticleText'].tolist()[0]
for i in range(len(rec_idx)):
    rec_str.append(recommd2012[rec_idx[i]:va_idx[i]])
    
recommd.loc[recommd['Year'] == 2012, 'ArticleText'] = ''.join(rec_str)

In [0]:
# fix 2004
# index for recommendation
rec_idx = []
prev = 0
for i in re.finditer(r'recommendation \d+ ', recommd[recommd['Year'] == 2004]['ArticleText'].tolist()[0]):
    curr = int(re.findall(r'\d+', i.group(0))[0])
    if curr>prev:
        rec_idx.append(i.span()[0])
        prev = curr
# ignore the match that has occured before

# index for the start of other content other than recommendations
va_idx = []
prev = 0
for i in re.finditer(r'\nva response ', recommd[recommd['Year'] == 2004]['ArticleText'].tolist()[0]):
    va_idx.append(i.span()[0])

In [0]:
print(len(rec_idx))
print(len(va_idx))

5
5


In [0]:
rec_str = []
recommd2004 = recommd[recommd['Year'] == 2004]['ArticleText'].tolist()[0]
for i in range(len(rec_idx)):
    rec_str.append(recommd2004[rec_idx[i]:va_idx[i]])
    
recommd.loc[recommd['Year'] == 2004, 'ArticleText'] = ''.join(rec_str)

In [0]:
# fix 2007
# index for recommendation
rec_idx = []
prev = 0
for i in re.finditer(r'recommendation \d+ ', recommd[recommd['Year'] == 2007]['ArticleText'].tolist()[0]):
    curr = int(re.findall(r'\d+', i.group(0))[0])
    if curr>prev:
        rec_idx.append(i.span()[0])
        prev = curr
# ignore the match that has occured before

# index for recommendation
va_idx = []
prev = 0
for i in re.finditer(r'\ndiscussion ', recommd[recommd['Year'] == 2007]['ArticleText'].tolist()[0]):
    va_idx.append(i.span()[0])

In [0]:
print(len(rec_idx))
print(len(va_idx))

13
13


In [0]:
rec_str = []
recommd2007 = recommd[recommd['Year'] == 2007]['ArticleText'].tolist()[0]
for i in range(len(rec_idx)):
    rec_str.append(recommd2007[rec_idx[i]:va_idx[i]])
    
recommd.loc[recommd['Year'] == 2007, 'ArticleText'] = ''.join(rec_str)

In [0]:
# filter out invalid recommendations
recommd = recommd[[i[0] not in [94, 113] for i in recommd['PageNumber']]]
recommd = recommd[[len(i)>5 for i in recommd['ArticleText']]]

In [0]:
recommd.head()

,ArticleIndex,Title,ArticleText,PageNumber,Year
0,8,recommendations,"based on the its research, collaboration, part...","[17, 18, 19]",1996
1,23,recommendations,the full committee recommends that the secreta...,"[67, 68, 69, 70, 71, 72, 73]",1997
2,37,statement of recommendations,\n1. the full committee recommends the secreta...,"[94, 95]",1998
3,39,recommendations,\n1. initiate a review of the funding and oper...,"[98, 99, 100]",1998
4,43,vas response to advisory committee recommendat...,\n1. initiate a review of the funding and oper...,"[113, 114, 115, 116, 117, 118, 119]",1998


In [0]:
# save data to delta lake
recommd_spark=spark.createDataFrame(recommd) 
recommd_spark.printSchema()
recommd_spark.show()

root
 |-- ArticleIndex: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- ArticleText: string (nullable = true)
 |-- PageNumber: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Year: long (nullable = true)

+------------+--------------------+--------------------+--------------------+----+
|ArticleIndex|               Title|         ArticleText|          PageNumber|Year|
+------------+--------------------+--------------------+--------------------+----+
|           8|     recommendations|based on the its ...|        [17, 18, 19]|1996|
|          23|     recommendations|the full committe...|[67, 68, 69, 70, ...|1997|
|          39|     recommendations|\n1. initiate a r...|       [98, 99, 100]|1998|
|          61|     recommendations|overview \n\nthe ...|[140, 141, 142, 1...|1999|
|          68|status of previou...|\noverview \n\nth...|[166, 167, 168, 169]|2000|
|          76|committee general...|\nlisted below ar...|     [222, 223, 224]|2002|
|   

In [0]:
recommd_spark.write \
        .mode("overwrite") \
        .format("delta") \
        .option("overwriteSchema", "true") \
        .saveAsTable('NAII.CMV_Reports_Article_Recommendations')

In [0]:
articleDF = spark.sql('select * from NAII.CMV_Reports_Article_Recommendations order by Year').toPandas()

In [0]:
def get_recomd_start_idx(text):
    rec_idx = []
    prev = 0
    for i in re.finditer(r'(recommendation \d+ |\n\d+\. |\nnumber \d+\. )', text):
        curr = int(re.findall(r'\d+', i.group(0))[0])
        if curr>prev:
            rec_idx.append(i.span()[0])
            prev = curr
    return rec_idx

In [0]:
recommds_list = {}
for y in articleDF['Year'].unique():
    recText = []
    text = articleDF[articleDF['Year'] == y]['ArticleText'].tolist()[0]
    ri = get_recomd_start_idx(text)
    ri.append(len(text))
    for i in range(len(ri)-1):
        recText.append(text[ri[i]:ri[i+1]])
    recommds_list[y] = recText    
    

In [0]:
# get the number of sub lists for each recommendation article
print([f'{key}: {len(val)}' for key, val in recommds_list.items()])

['1996: 16', '1997: 62', '1998: 10', '1999: 4', '2000: 1', '2001: 5', '2002: 4', '2004: 5', '2005: 9', '2006: 10', '2007: 13', '2008: 22', '2009: 14', '2011: 13', '2012: 11', '2013: 5', '2014: 6', '2015: 4', '2016: 3', '2017: 4', '2018: 4', '2020: 4']


In [0]:
# capture recommendation lists in 1999
def get_recomd_1999():
    text = articleDF[articleDF['Year'] == 1999]['ArticleText'].tolist()[0]
    rec_idx = []
    prev = 0
    for i in re.finditer(r'\nnumber \d+\.', text):
        curr = int(re.findall(r'\d+', i.group(0))[0])
        if curr>prev:
            rec_idx.append(i.span()[0])
            prev = curr
    recText = []
    rec_idx.append(len(text))
    for i in range(len(rec_idx)-1):
        recText.append(text[rec_idx[i]:rec_idx[i+1]])
    return recText
recommds_list[1999] = get_recomd_1999()

In [0]:
# capture recommendation lists in 2000
def get_recomd_2000():
    text = articleDF[articleDF['Year'] == 2000]['ArticleText'].tolist()[0]
    rec_idx = []
#     prev = 0
    for i in re.finditer(r'(\nnumber ?(\d+)?\.?|\nrecommends that)', text):
#         curr = int(re.findall(r'\d+', i.group(0))[0])
#         if curr>prev:
        rec_idx.append(i.span()[0])
#             prev = curr
    recText = []
    rec_idx.append(len(text))
    for i in range(len(rec_idx)-1):
        recText.append(text[rec_idx[i]:rec_idx[i+1]])
    return recText
recommds_list[2000] = get_recomd_2000()
# get_recomd_2000()

In [0]:
# capture recommendation lists in 2002
def get_recomd_2002():
    text = articleDF[articleDF['Year'] == 2002]['ArticleText'].tolist()[0]
    rec_idx = []
#     prev = 0
    for i in re.finditer(r'\n\d+\.', text):
#         curr = int(re.findall(r'\d+', i.group(0))[0])
#         if curr>prev:
        rec_idx.append(i.span()[0])
#             prev = curr
    recText = []
    rec_idx.append(len(text))
    for i in range(len(rec_idx)-1):
        recText.append(text[rec_idx[i]:rec_idx[i+1]])
    return recText
recommds_list[2002] = get_recomd_2002()

In [0]:
# check again
print([f'{key}: {len(val)}' for key, val in recommds_list.items()])

['1996: 16', '1997: 62', '1998: 10', '1999: 14', '2000: 15', '2001: 5', '2002: 10', '2004: 5', '2005: 9', '2006: 10', '2007: 13', '2008: 22', '2009: 14', '2011: 13', '2012: 11', '2013: 5', '2014: 6', '2015: 4', '2016: 3', '2017: 4', '2018: 4', '2020: 4']


In [0]:
# recommds_list

In [0]:
# spell correction

for key in recommds_list:
    val = [remove_extra_space(remove_new_line(i)).split(' ') for i in recommds_list[key]]
    corrected = []
    for v in val:
        recommdCrcted = ' '.join([sym_spell.lookup(t, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True, ignore_token=r"(\w+\d|\W|\d|va|\w+\W)")[0].term for t in v if t!= ''])
        if recommdCrcted.strip()[-1] != '.':
             recommdCrcted += '.'
        corrected.append(recommdCrcted)
                
    recommds_list[key] = corrected

In [0]:
recommds_list[1996]

Out[147]: ['1. establish a separate budget line item for the advisory committee on minority veterans.',
 '2. appropriate 125,000.00 for the fiscal year year budget for the advisory committee on minority veterans.',
 '3. recognize the appropriate regional and national minority organizations that serve our country diverse population groups and invite them as genuine partners, consultants, advisors and expert witnesses in enriching the departments commitment to serve the needs of our minority veterans.',
 '4. authorize an assessment be conducted and a report submitted in by 97 by the department of veterans affairs to determine the validity of the committees concern regarding ethnic representation on the rating panels and the men review boards, as well as among the counselor who advise our minority veteran populations.',
 '5. establish and fund an intro agency task force, chaired by the director of the center for minority veterans, to conduct a needs assessment for health care, benefits an

In [0]:
# save recommds_list as a dataframe
recommdSents = pd.DataFrame({'Year': [i for i in recommds_list],
                            'Recommendations': [remove_extra_space(remove_backslash(remove_new_line('&&'.join(i)))) for i in recommds_list.values()]})

In [0]:
recommdSents_spark=spark.createDataFrame(recommdSents) 
recommdSents_spark.printSchema()
recommdSents_spark.show()

root
 |-- Year: long (nullable = true)
 |-- Recommendations: string (nullable = true)

+----+--------------------+
|Year|     Recommendations|
+----+--------------------+
|1996|1. establish a se...|
|1997|1. establish a ta...|
|1998|1. initiate a rev...|
|1999|number 1. acting ...|
|2000|number 3. recogni...|
|2001|number 1. revital...|
|2002|1. recommends the...|
|2004|recommendation 1 ...|
|2005|1. san juan vamc ...|
|2006|1. outreach progr...|
|2007|recommendation 1 ...|
|2008|recommendation 1 ...|
|2009|1. promote cultur...|
|2011|1. establish hiri...|
|2012|recommendation 1 ...|
|2013|recommendation 1 ...|
|2014|recommendation 1 ...|
|2015|recommendation 1 ...|
|2016|recommendation 1 ...|
|2017|recommendation 1 ...|
+----+--------------------+
only showing top 20 rows



In [0]:
recommdSents_spark.write \
        .mode("overwrite") \
        .format("delta") \
        .option("overwriteSchema", "true") \
        .saveAsTable('NAII.CMV_Reports_Article_Recommendations_lists')